In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import LabelEncoder

In [2]:
def read_data_and_process():
    with open("data.pkl", "rb") as file:
        df = pickle.load(file)
        
    lbe_user = LabelEncoder()
    df["personId"] = lbe_user.fit_transform(df["personId"])
    
    lbe_item = LabelEncoder()
    df["contentId"] = lbe_item.fit_transform(df["contentId"])
   
    df["eventStrength"] = df["eventStrength"].astype(np.float32)
    print(df)
    return df, lbe_user, lbe_item    

In [3]:
class ShuffleDataIterator(object):
    """
    随机生成batch数据
    """
    def __init__(self, inputs, batch_size=10):
        self.inputs = inputs
        self.batch_size = batch_size
        self.num_cols = len(self.inputs)
        self.len = len(self.inputs[0])
        self.inputs = np.transpose(np.vstack([np.array(self.inputs[i]) for i in range(self.num_cols)]))
    
    def __len__(self):
        return self.len
    
    def __iter__(self):
        return self #实例本身就是迭代对象
    
    def __next__(self):
        return self.next()
    
    def next(self):
        idx = np.random.randint(0,self.len,(self.batch_size,))
        out = self.inputs[idx,:]
        return [out[:, i] for i in range(self.num_cols)]       

In [4]:
class OneEpochDataIterator(ShuffleDataIterator):
    """
    顺序生成batch数据，测试集用
    """
    def __init__(self, inputs, batch_size=10):
        super(OneEpochDataIterator, self).__init__(inputs, batch_size=batch_size)
        if batch_size > 0:
            self.idx_group = np.array_split(np.arange(self.len), self.len / np.ceil(self.len / batch_size))
        else:
            self.idx_group = [np.arange(self.len)]
        self.group_id =0
    
    def next(self):
        if self.group_id >= len(self.idx_group):
            self.group_id = 0
            raise StopIteration
        out = self.inputs[self.idx_group[self.group_id], :]
        self.group_id += 1
        return [out[:,i] for i in range(self.num_cols)]    

In [5]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

#使用矩阵分解搭建网络结构
def svd_model(user_batch, item_batch, user_num, item_num, dim=5, device="/cpu:0"):
    #使用cpu
    with tf.device("/cpu:0"):
        #初始化bias项
        global_bias = tf.get_variable("global_bias", shape=[])
        w_bias_user = tf.get_variable("embd_bias_user", shape=[user_num])
        w_bias_item = tf.get_variable("embd_bias_item", shape=[item_num])
        #bias向量
        bias_user = tf.nn.embedding_lookup(w_bias_user, user_batch, name="bias_user")
        bias_item = tf.nn.embedding_lookup(w_bias_item, item_batch, name="bias_item")
        
        w_user = tf.get_variable("embd_user", shape=[user_num, dim],
                                initializer=tf.truncated_normal_initializer(stddev=0.02))
        w_item = tf.get_variable("embd_item", shape=[item_num, dim],
                                initializer=tf.truncated_normal_initializer(stddev=0.02))
        #user向量与item向量
        embd_user = tf.nn.embedding_lookup(w_user, user_batch, name="embedding_user")
        embd_item = tf.nn.embedding_lookup(w_item, item_batch, name="embedding_item")
    
    with tf.device(device):
        #对user向量和item向量求内积
        infer = tf.reduce_sum(tf.multiply(embd_user, embd_item), axis=1)
        #加上几个偏置项
        infer = tf.add(infer, global_bias)
        infer = tf.add(infer, bias_user)
        infer = tf.add(infer, bias_item, name="svd_inference")
        #加上正则化项
        regularizer = tf.add(tf.nn.l2_loss(embd_user), tf.nn.l2_loss(embd_item), name="svd_inference")
    return infer, regularizer     
        

Instructions for updating:
non-resource variables are not supported in the long term


In [6]:
#模型优化
def optimization(infer, regularizer, rate_batch, learning_rate= 0.001, reg=0.1, device="/cpu:0"):
    global_step = tf.train.get_global_step()
    assert global_step is not None
    #选择合适的optimizer做优化
    with tf.device(device):
        cost_l2 = tf.nn.l2_loss(tf.subtract(infer, rate_batch))
        penalty = tf.constant(reg, dtype=tf.float32, shape=[], name="l2")
        cost = tf.add(cost_l2, tf.multiply(regularizer, penalty))
        train_op = tf.train.AdamOptimizer(learning_rate).minimize(cost, global_step=global_step)
    return cost, train_op    

In [7]:
from six import next
from tensorflow.core.framework import summary_pb2

np.random.seed(42)

df,_,_ = read_data_and_process()
#一批数据大小
BATCH_SIZE = 256
#用户数
USER_NUM = len(df.groupby("personId").size())
print(USER_NUM)
#文章数
ITEM_NUM = len(df.groupby("contentId").size())
print(ITEM_NUM)
#维度
DIM = 15
#最大迭代轮数
EPOCH_MAX = 100
#提前停止训练设置
PATIENT = 3
#使用cpu训练
DEVICE = "/cpu:0"

#截断
def clip(x):
    return np.clip(x, 1.0, 5.0)

#summary
def make_scalar_summary(name, val):
    return summary_pb2.Summary(value=[summary_pb2.Summary.Value(tag=name, simple_value=val)])

#获取数据
def get_data(df):
    rows = len(df)
    df = df.iloc[np.random.permutation(rows)].reset_index(drop=True)
    split_index = int(rows*0.9)
    df_train = df[0:split_index]
    df_test = df[split_index:].reset_index(drop=True)
    print(df_train.shape, df_test.shape)
    return df_train, df_test

       personId  contentId  eventStrength
0             0         65       1.000000
1             0        160       1.000000
2             0        188       1.000000
3             0        196       1.000000
4             0        313       3.169925
...         ...        ...            ...
39101      1139       2858       3.247927
39102      1139       2864       1.000000
39103      1139       2919       1.000000
39104      1139       2922       1.000000
39105      1139       2978       1.000000

[39106 rows x 3 columns]
1140
2984


In [8]:
#训练模型
import time
from collections import deque

def train_model(train, test):
    samples_per_batch = len(train) #batch_size
    #分批生成训练数据
    iter_train = ShuffleDataIterator([train["personId"],
                                      train["contentId"],
                                      train["eventStrength"]],
                                      batch_size=BATCH_SIZE)
    #分批生成测试数据
    iter_test = OneEpochDataIterator([test["personId"],
                                      test["contentId"],
                                      test["eventStrength"]],
                                      batch_size=-1)
    #user和item batch
    #tf.compat.v1.disable_eager_execution()
    user_batch = tf.placeholder(tf.int32, shape=[None], name="id_user")
    item_batch = tf.placeholder(tf.int32, shape=[None], name="id_item")
    rate_batch = tf.placeholder(tf.float32, shape=[None])
    
    #构建graph和训练
    infer, regularizer = svd_model(user_batch, item_batch, user_num=USER_NUM, item_num=ITEM_NUM, device=DEVICE)
    global_step = tf.train.get_or_create_global_step()
    _, train_op = optimization(infer, regularizer,rate_batch, learning_rate=0.001, reg=0.1, device=DEVICE)
    
    #初始化所有变量
    init_op = tf.global_variables_initializer()
    
    #开始迭代
    with tf.Session() as sess:
        sess.run(init_op)
        summary_writer = tf.summary.FileWriter(logdir="/tmp/svd/log", graph=sess.graph)
        print("{} {} {} {}".format("epoch", "train_error", "val_error", "elapsed_time"))
        errors = deque(maxlen=samples_per_batch)
        start = time.time()
        for i in range(EPOCH_MAX * samples_per_batch):
            users, items, rates = next(iter_train)
            _, pred_batch = sess.run([train_op, infer], feed_dict={user_batch:users,
                                                                   item_batch:items,
                                                                   rate_batch:rates})
            pred_batch = clip(pred_batch)
            errors.append(np.power(pred_batch - rates, 2))
            min_test_err = np.float("inf")
            patient = 0
            if i % samples_per_batch == 0:
                train_err = np.sqrt(np.mean(errors))
                test_err_temp = np.array([])
                for users, items, rates in iter_test:
                    pred_batch = sess.run(infer, feed_dict={user_batch: users, item_batch: items})
                    pred_batch = clip(pred_batch)
                    test_err_temp = np.append(test_err_temp, np.power(pred_batch - rates, 2))
                end = time.time()
                test_err = np.sqrt(np.mean(test_err_temp))
                if test_err < min_test_err:
                    min_test_err = test_err  
                    best_infer = infer
                    patient = 0
                else:
                    patient +=1
                    if patient > PATIENT: #如果超过3轮损失不再减小，提前结束训练
                        break
                print("{:3d}  {:f}  {:f}  {:f} (s)".format(i // samples_per_batch, train_err, test_err, end - start))
                train_err_summary = make_scalar_summary("training_error", train_err)
                test_err_summary = make_scalar_summary("test_error", test_err)
                summary_writer.add_summary(train_err_summary, i)
                summary_writer.add_summary(test_err_summary, i)
                start = end

In [9]:
#获取数据
df_train, df_test = get_data(df)

(35195, 3) (3911, 3)


In [10]:
#训练模型
train_model(df_train, df_test)

epoch train_error val_error elapsed_time
  0 0.681453 0.720151 1.577187(s)
  1 0.446098 0.651888 144.899901(s)
  2 0.413603 0.652893 137.434732(s)
  3 0.410387 0.653014 152.257449(s)
  4 0.408676 0.652223 154.831831(s)
  5 0.407818 0.653215 123.572953(s)
  6 0.406921 0.651391 160.825494(s)
  7 0.406973 0.651359 141.359723(s)
  8 0.406623 0.653182 136.859840(s)
  9 0.406402 0.651519 140.825263(s)
 10 0.406439 0.650973 151.570783(s)
 11 0.406500 0.650476 141.823403(s)
 12 0.406347 0.652210 125.274977(s)
 13 0.406429 0.651357 138.049936(s)
 14 0.406511 0.651460 151.709136(s)
 15 0.406230 0.652179 157.513473(s)
 16 0.406220 0.651793 150.004509(s)
 17 0.406221 0.653788 126.048562(s)
 18 0.406282 0.653857 134.717863(s)
 19 0.406363 0.653688 136.576513(s)
 20 0.406444 0.653064 132.477874(s)
 21 0.406135 0.653975 125.216912(s)
 22 0.406437 0.656467 136.697678(s)
 23 0.406358 0.653361 147.748073(s)
 24 0.406144 0.656972 128.289783(s)
 25 0.406477 0.656000 128.347795(s)
 26 0.406345 0.657256 131